In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('интерполяция.csv', sep=';', index_col=0, engine='python', encoding='utf-8')
del df['Unnamed: 0.1']

In [3]:
def get_nans_intervals(v):
    # это слишком гениально, чтобы объяснять
    # возвращает все интервалы из идущих подряд нанов в векторе
    m = np.concatenate(([True], ~pd.isnull(v), [True]))
    return np.flatnonzero(m[1:] != m[:-1]).reshape(-1, 2)

In [4]:
def get_len(interval):
    return interval[1] - interval[0]

In [5]:
# чистим те, что без данных за весь период

features_to_check = list(df)[18:-3]

indexes_to_drop = set()
for col_name in features_to_check:
    for i in df['index'].unique():
        # если банк уже в списке на удаление, проверять его не имеет смысла
        if i not in indexes_to_drop:
            column = df[df['index'] == i][col_name]
            nan_intervals = get_nans_intervals(column)
            # банк нужно удалить, если есть всего один интервал из nan'ов и он по длине равен длине самой колонки
            if len(nan_intervals == 1) and get_len(nan_intervals[0]) == len(column):
                indexes_to_drop.add(i)

In [6]:
len(indexes_to_drop)

110

In [7]:
df = df[~df['index'].isin(indexes_to_drop)]

In [96]:
# for col_name in features_to_check:
#     print(get_nans_intervals(df[df['index'] == 97][col_name]))

[[ 0  3]
 [11 12]]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[[ 0  3]
 [ 9 12]]
[[ 0  3]
 [ 9 12]]
[[ 0  3]
 [ 9 12]]


In [97]:
# index_mask = (df['index'] == 97)
# for col_name in features_to_check:
#     column = df[index_mask][col_name]
#     for nan_interval in get_nans_intervals(column):
#         if get_len(nan_interval) <= 4:
#             if 0 in nan_interval:
#                 df.loc[index_mask, col_name] = df[index_mask][col_name].bfill()
#             elif len(column) in nan_interval:
#                 df.loc[index_mask, col_name] = df[index_mask][col_name].ffill()

In [98]:
# for col_name in features_to_check:
#     print(get_nans_intervals(df[df['index'] == 97][col_name]))

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [8]:
row_indexes_to_drop = []
for i in df['index'].unique():
    index_mask = (df['index'] == i)
    for col_name in features_to_check:
        column = df[index_mask][col_name]
        nan_intervals = get_nans_intervals(column)
        for nan_interval in nan_intervals:
            if get_len(nan_interval) <= 4:
                # если в nan_interval содержится 0, значит, есть кусок пропусков в начале
                if 0 in nan_interval:
                    df.loc[index_mask, col_name] = column.bfill()
                # аналогично с len(column) и концом
                elif len(column) in nan_interval:
                    df.loc[index_mask, col_name] = column.ffill()
            # если длина пропуска больше 4 и банк действующий, добавляем индексы nan'ов в список на удаление
            elif df[index_mask].iloc[0].status:
                row_indexes_to_drop.extend(df[index_mask].index[nan_interval[0]:nan_interval[1]])
                
                
# Печатает индексы и колонки, в которых есть пропуски посередине        
#         try:
#             assert len(nan_intervals) < 3
#             if len(nan_intervals) == 1:
#                 assert 0 in nan_intervals[0] or len(df[df['index'] == i][col]) in nan_intervals[0]
#             elif len(nan_intervals) == 2:
#                 assert 0 in nan_intervals[0] and len(df[df['index'] == i][col]) in nan_intervals[1]
#         except AssertionError:
#             print(i, col_name, len(column), nan_intervals)

In [9]:
len(row_indexes_to_drop)

9053

In [10]:
df = df[~df.index.isin(row_indexes_to_drop)]
df.shape

(45861, 45)

In [38]:
df.to_csv('final.csv', sep=';', index=None)

In [11]:
df4 = df.groupby(['index', 'year', 'month']).apply(lambda x: x.isna().sum()*100/len(x))

In [13]:
df4.to_csv('fin.csv', sep=';', encoding='cp1251')